## Loading Data

In [1]:
import pandas as pd
df = pd.read_csv('constraint_category_initial_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   dataset                    60 non-null     object
 1   instruction                60 non-null     object
 2   code                       60 non-null     object
 3   test                       20 non-null     object
 4   Characteristics            60 non-null     object
 5   constraints                60 non-null     object
 6   Characteristics_List       60 non-null     object
 7   Mapped_Characteristics     60 non-null     object
 8   Mapped_Constraints         60 non-null     object
 9   Mapped_Characteristics_v2  60 non-null     object
 10  Mapped_Constraints_v2      60 non-null     object
 11  categories_v1              60 non-null     object
 12  categories_v2              60 non-null     object
 13  Manual_categories          60 non-null     object
 14  LLM_categori

## Loading the model 

In [2]:
# base_url="https://inference-3scale-apicast-production.apps.rits.fmaas.res.ibm.com/pixtral-large-instruct-2411/v1"

In [3]:
from openai import OpenAI

class RITSClient:

    def __init__(self, api_key, base_url="https://inference-3scale-apicast-production.apps.rits.fmaas.res.ibm.com/granite-4-0-medium-franconia/v1"):
        llm = OpenAI(
        api_key=api_key,  
        base_url=base_url,
        default_headers={'RITS_API_KEY': api_key}
        )
        self.llm = llm

    def get_model_response(self,prompt,system_prompt="You are a helpful assistant.", model_id="ibm-granite/granite-4.0-medium-prerelease-franconia.r250523a", max_new_tokens=400, temperature=0):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]

        response = self.llm.chat.completions.create(
            model=model_id,
            messages=messages,
            max_tokens=max_new_tokens,
            temperature=temperature,

        )

        return response.choices[0].message.content.strip()

In [4]:
import os
import dotenv
dotenv.load_dotenv() 
api_key = os.environ.get('RITS_API_KEY')

In [5]:
client = RITSClient(api_key,base_url="https://inference-3scale-apicast-production.apps.rits.fmaas.res.ibm.com/granite-3-3-8b-instruct/v1")
# Example usage
prompt = "What is the capital of France?"
response = client.get_model_response(prompt, model_id="ibm-granite/granite-3.3-8b-instruct")
print(response)  # Should print the model's response to the question

The capital of France is Paris.


In [6]:
client2 = RITSClient(api_key)
response2 = client2.get_model_response(prompt)
print(response2)  

The capital of France is Paris.


# Utility Functions

In [7]:
import json

def extract_json(string,col_name="Constraints"):
    try:
        json_string = string.strip().replace('```json\n', '', 1).replace('\n```', '', 1)
        constraint_json = json.loads(json_string)
        return constraint_json.get(col_name, [])
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e} in string: {string}")
        return []
    except AttributeError as e:
        print(f"Attribute error: {e} in string: {string}")
        return []

# Step 1- Define Constraint Catgeories

In [8]:
categories = [
    "Code Structure and Modularity",
    "Input and Output Handling",
    "Error Handling and Robustness",
    "Data Processing and Transformation",
    "Performance and Optimization",
    "Library and API Usage",
    "Testing and Debugging",
    "Documentation and Readability",
    "Security and Privacy",
    "Reproducibility and Consistency",
    "Mathematical Computation",
    "File and Data Management",
    "UI and Interaction",
]
categories_str = "\n".join(categories)

# Step 2 : Relevant Category Selection

In [9]:
SYSTEM_PROMPT_STAGE1 = """
You are a categorization expert for programming tasks. Your goal is to accurately map a programming instruction and its associated code to all applicable high-level constraint categories from a predefined list.
"""

def get_prompt_stage1_select_categories(instruction: str, code: str, all_constraint_categories: list[str]) -> str:
    categories_str = "\n".join(all_constraint_categories)
    prompt = f"""
    "task": "Classify the natural language `instruction` and its corresponding `code` into all relevant high-level constraint categories from the provided comprehensive list.",

    "context": "You are given an `instruction` that describes a coding task, `code` for context, and a list of `all_constraint_categories`.",

    "goal": "Select **all** relevant categories from the `all_constraint_categories` list that apply directly, indirectly, or even potentially to the given `instruction` and `code`.
    For each category, think carefully about both explicit requirements and implicit expectations that might arise in real-world programming tasks.
    If there is **any plausible reason**—whether due to the instruction's wording, the code's structure, edge cases, potential extensions, or real-world usage scenarios—that the category might impose constraints, then **include it**.

    Be exhaustive on the side of inclusion. The goal is to **maximize coverage** of all semantically or practically relevant constraint categories, not to limit selection to a minimal subset.

    Your output must be a JSON object containing a single field: 'relevant_categories'.
    ",

    "JSON Response Format": {{
        "relevant_categories": [
            "List of selected relevant categories from 'all_constraint_categories'. Each string should be the exact name of the matching supercategory."
        ]
    }},

    "Inputs Required": {{
        "instruction": {instruction}
        "code": {code}
        "all_constraint_categories": {categories_str}
    }}
    """
    return prompt
prompt = get_prompt_stage1_select_categories(df.iloc[26]["instruction"],df.iloc[26]["code"],categories)
print(prompt)


    "task": "Classify the natural language `instruction` and its corresponding `code` into all relevant high-level constraint categories from the provided comprehensive list.",

    "context": "You are given an `instruction` that describes a coding task, `code` for context, and a list of `all_constraint_categories`.",

    "goal": "Select **all** relevant categories from the `all_constraint_categories` list that apply directly, indirectly, or even potentially to the given `instruction` and `code`.
    For each category, think carefully about both explicit requirements and implicit expectations that might arise in real-world programming tasks.
    If there is **any plausible reason**—whether due to the instruction's wording, the code's structure, edge cases, potential extensions, or real-world usage scenarios—that the category might impose constraints, then **include it**.

    Be exhaustive on the side of inclusion. The goal is to **maximize coverage** of all semantically or practical

In [10]:
print(df.iloc[26]["instruction"])

Perform a chi-square test of independence of variables in a contingency table. This function takes a DataFrame containing categorical data and two column names, then constructs a contingency table from the two categorical columns and performs a chi-square test of independence. It returns the p-value of the test, which indicates the probability of observing the data if the null hypothesis (independence of the variables) is true. >>> np.random.seed(42) >>> data = pd.DataFrame({ ...     'a': np.random.choice(['A', 'B'], size=100), ...     'b': np.random.choice(['X', 'Y'], size=100) ... }) >>> task_func(data, 'a', 'b') 1.0
The function should raise the exception for: ValueError: If 'data' is empty, if 'col1' or 'col2' are not in 'data', if one or both of the columns do not have multiple categories, or if some categories have less than 5 observations (violating the chi-square test assumptions). TypeError: If one or both of the columns contain non-categorical data.
The function should output

In [11]:
relevant_categories = client2.get_model_response(prompt=prompt, system_prompt=SYSTEM_PROMPT_STAGE1)
print(relevant_categories)
relevant_categories = extract_json(relevant_categories,"relevant_categories")
print(relevant_categories)

```json
{
    "relevant_categories": [
        "Input and Output Handling",
        "Error Handling and Robustness",
        "Data Processing and Transformation",
        "Library and API Usage",
        "Mathematical Computation",
        "Code Structure and Modularity"
    ]
}
```

**Justification:**

- **Input and Output Handling**: The function takes a DataFrame and two column names as input and returns a float (p-value) as output.
- **Error Handling and Robustness**: The function includes checks for empty DataFrame, non-existent columns, non-categorical data, single category, and small counts in categories, raising appropriate exceptions.
- **Data Processing and Transformation**: The function constructs a contingency table from the input DataFrame and columns.
- **Library and API Usage**: The function uses pandas for DataFrame manipulation and scipy.stats for the chi-square test.
- **Mathematical Computation**: The function performs a chi-square test of independence.
- **Code Stru

# Step 3: Instruction Simplification & Implicit Constraint Extraction

In [12]:
SYSTEM_PROMPT_STAGE2 = """
You are an expert natural language processor and code instruction analyst. Your task is to meticulously read a programming instruction, identify and extract any embedded constraints, separate them from the core problem description, and map them to provided high-level categories. You must preserve any provided starter code or solution snippets.
"""

def get_prompt_stage2_extract_and_simplify(instruction: str, relevant_categories: list[str]) -> str:
    relevant_categories_str = ", ".join(relevant_categories)

    prompt = f"""
    "task": "Carefully analyze the provided programming `instruction`. Your main goal is to separate the core problem description from any explicit constraints or directives embedded within the instruction's text. You must return the core problem description as 'simplified_instruction' and all identified explicit constraints as 'extracted_constraints'.",

    "context": "You are provided with an `instruction` that describes a coding task and a list of `relevant_categories` that apply to this instruction. The `instruction` might contain specific rules, formatting requirements, or implementation details interwoven into its narrative. You need to pull these out.",

    "goal": "Perform the following steps:
    1. **Identify and Extract Explicit Constraints:** Read the `instruction` and pinpoint all **explicit** constraints, directives, or specific requirements (e.g., about function names, return types, error handling, algorithms to use, variable naming, documentation, etc.).
    2. **Split Compound Constraints into Atomic Ones:** If any sentence or clause contains multiple distinct requirements, split it into **multiple entries**, ensuring **each extracted constraint refers to exactly one requirement or condition**.
       Do not combine multiple checks (e.g., multiple `if` conditions or causes for `ValueError`) into one constraint.
    3. **Simplify Instruction:** Create a 'simplified_instruction' by removing only these identified explicit constraints, leaving behind the core problem explanation.
        **CRITICAL:** If the original `instruction` contains any starter code (e.g., code in ```python blocks) or any solution snippets demarcated by 'BEGIN SOLUTION', you **MUST NOT** modify or remove these from the 'simplified_instruction'.
        **Important:** If the original instruction contains any explanation or examples that are not constraints, you should keep them in the 'simplified_instruction' as they are part of the core problem description.
    4. **Format Extracted Constraints with Categories:** For each extracted constraint:
        - Include a separate item in the 'extracted_constraints' list.
        - Set its 'type' to one of the categories from `relevant_categories`.
        - Set 'instruction_part' to 'Extracted from instruction'.

    Example:
    - Original compound constraint: "Raise ValueError if input is None or if it's an empty list."
    - Extracted as:
        {{
            "type": "Error Handling and Robustness",
            "constraint": "Raise ValueError if input is None.",
            "instruction_part": "Extracted from instruction"
        }},
        {{
            "type": "Error Handling and Robustness",
            "constraint": "Raise ValueError if input is an empty list.",
            "instruction_part": "Extracted from instruction"
        }}

    Your output must be a JSON object with two fields: 'simplified_instruction' and 'extracted_constraints'.
    ",

    "JSON Response Format": {{
        "simplified_instruction": "The core problem description from the original instruction, with explicit constraints/directives removed. Starter code (e.g., ```python blocks) and solution snippets (e.g., marked by 'BEGIN SOLUTION') must be preserved.",
        "extracted_constraints": [
            {{
                "type": "Constraint_Category_Name",  // MUST be one of the provided relevant_categories
                "constraint": "A single, atomic extracted constraint.",
                "instruction_part": "Extracted from instruction"
            }}
        ]
    }},

    "Inputs Required": {{
        "instruction": {instruction}
        "relevant_categories": {relevant_categories_str}
    }}
    """
    return prompt


prompt_stage2 = get_prompt_stage2_extract_and_simplify(df.iloc[26]["instruction"],relevant_categories)
print(prompt_stage2)


    "task": "Carefully analyze the provided programming `instruction`. Your main goal is to separate the core problem description from any explicit constraints or directives embedded within the instruction's text. You must return the core problem description as 'simplified_instruction' and all identified explicit constraints as 'extracted_constraints'.",

    "context": "You are provided with an `instruction` that describes a coding task and a list of `relevant_categories` that apply to this instruction. The `instruction` might contain specific rules, formatting requirements, or implementation details interwoven into its narrative. You need to pull these out.",

    "goal": "Perform the following steps:
    1. **Identify and Extract Explicit Constraints:** Read the `instruction` and pinpoint all **explicit** constraints, directives, or specific requirements (e.g., about function names, return types, error handling, algorithms to use, variable naming, documentation, etc.).
    2. **Spl

In [13]:
instruction2 = "Write a function that sorts a list of numbers using merge sort with time complexity O(n log n)."
code2 = "def merge_sort(arr):\n    if len(arr) > 1:\n        mid = len(arr) // 2\n        left_half = arr[:mid]\n        right_half = arr[mid:]\n\n        merge_sort(left_half)\n        merge_sort(right_half)\n\n        i = j = k = 0\n\n        while i < len(left_half) and j < len(right_half):\n            if left_half[i] < right_half[j]:\n                arr[k] = left_half[i]\  i += 1\n            else:\n                arr[k] = right_half[j]\  j += 1\n            k += 1\n\n        while i < len(left_half):\n            arr[k] = left_half[i]\  i += 1\  k += 1\n\n        while j < len(right_half):\n            arr[k] = right_half[j]\  j += 1\  k += 1\n\n    return arr"
example_prompt2 = get_prompt_stage2_extract_and_simplify(instruction2,relevant_categories)
print(example_prompt2)


    "task": "Carefully analyze the provided programming `instruction`. Your main goal is to separate the core problem description from any explicit constraints or directives embedded within the instruction's text. You must return the core problem description as 'simplified_instruction' and all identified explicit constraints as 'extracted_constraints'.",

    "context": "You are provided with an `instruction` that describes a coding task and a list of `relevant_categories` that apply to this instruction. The `instruction` might contain specific rules, formatting requirements, or implementation details interwoven into its narrative. You need to pull these out.",

    "goal": "Perform the following steps:
    1. **Identify and Extract Explicit Constraints:** Read the `instruction` and pinpoint all **explicit** constraints, directives, or specific requirements (e.g., about function names, return types, error handling, algorithms to use, variable naming, documentation, etc.).
    2. **Spl

In [14]:
response = client2.get_model_response(example_prompt2,SYSTEM_PROMPT_STAGE2)
print(response)

{
  "simplified_instruction": "Write a function that sorts a list of numbers using merge sort.",
  "extracted_constraints": [
    {
      "type": "Time Complexity",
      "constraint": "with time complexity O(n log n)",
      "instruction_part": "Extracted from instruction"
    }
  ]
}


# Testing on NL instructions

## Complex bench data

In [15]:
!wget https://raw.githubusercontent.com/thu-coai/ComplexBench/refs/heads/main/data/data_final.json

--2025-07-02 21:48:53--  https://raw.githubusercontent.com/thu-coai/ComplexBench/refs/heads/main/data/data_final.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6743190 (6.4M) [text/plain]


Saving to: ‘data_final.json.1’

data_final.json.1   100%[===================>]   6.43M  --.-KB/s    in 0.09s   

2025-07-02 21:49:05 (68.7 MB/s) - ‘data_final.json.1’ saved [6743190/6743190]



In [16]:
import pandas as pd
complex_bench = pd.read_json("data_final.json")
print(complex_bench.columns)


Index(['main_id', 'group', 'idx_in_group', 'instruction', 'instruction_en',
       'task_types', 'constraint_dimensions', 'composition_types', 'category',
       'scoring_questions'],
      dtype='object')


## FollowBench Data

In [17]:
from datasets import load_dataset

followbench_data = load_dataset("YuxinJiang/FollowBench")

In [18]:
followbench_data = followbench_data["train"].to_pandas()
print(followbench_data.columns)

Index(['example_id', 'category', 'source', 'instruction', 'level', 'target'], dtype='object')


In [19]:
followbench_data.head()

,example_id,category,source,instruction,level,target
0,1,content,t0_zsnoopt_data,Pick one category for the following text. The ...,0,
1,1,content,t0_zsnoopt_data,Identify one category from the list below for ...,1,
2,1,content,t0_zsnoopt_data,Identify one category and the sentiment convey...,2,
3,1,content,t0_zsnoopt_data,Analyze the provided text to pinpoint a catego...,3,
4,1,content,t0_zsnoopt_data,Analyze the supplied text to discern a categor...,4,


In [20]:
complex_bench.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150 entries, 0 to 1149
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   main_id                1150 non-null   int64 
 1   group                  1150 non-null   object
 2   idx_in_group           1150 non-null   int64 
 3   instruction            1150 non-null   object
 4   instruction_en         1150 non-null   object
 5   task_types             1150 non-null   object
 6   constraint_dimensions  1150 non-null   object
 7   composition_types      1150 non-null   object
 8   category               1150 non-null   object
 9   scoring_questions      1150 non-null   object
dtypes: int64(2), object(8)
memory usage: 90.0+ KB


In [21]:
complex_bench.head()

,main_id,group,idx_in_group,instruction,instruction_en,task_types,constraint_dimensions,composition_types,category,scoring_questions
0,0,complex_instruction_eval_10,1,"背景资料：\n""""""\n神顶科技（南京）有限公司（以下简称“神顶科技”）近日完成数千万元新一...","Background:\n""""\nShending Technology (Nanjing)...",Practical Writing,"[Consistency, Supportiveness, Template, Helpfu...",[And],And,"[{'point_id': 0, 'question': '模型回复是否符合指令中的格式要求..."
1,1,complex_instruction_eval_78,1,你是一个rapper，帮我写一首歌词，歌词第一句是“总是跟不上脚步”。歌词整体要正面、积极向...,"You are a rapper, help me write a song lyric, ...",Creative Writing,"[Language Style, Consistency, Helpfulness, Tar...",[And],And,"[{'point_id': 0, 'question': '模型回复是否帮助用户写了一首歌词..."
2,2,complex_instruction_eval_150,1,你是一位女性英语口语对话教练，名字是Echo，参考[角色]中的信息，和用户进行英文对话练习。...,You are a female English conversation coach na...,Task-oriented Role Play,"[Personalization, Language Style, Consistency,...",[And],And,"[{'point_id': 0, 'question': '模型回复是否代入了Echo角色，..."
3,3,complex_instruction_eval_190,1,"背景资料：\n""""""\n中大功率电源管理芯片公司源微半导体完成千万级A轮融资，由无锡海创投资...","Background:\n""""\nYuanwei Semiconductor, a comp...",Practical Writing,"[Template, Consistency, Helpfulness, Supportiv...",[And],And,"[{'point_id': 0, 'question': '模型是否完成了新闻改写？', '..."
4,4,complex_instruction_eval_201,1,人们因技术发展得以更好地掌控时间，但也有人因此成了时间的仆人。\n 这句话引发了你怎样的...,People are able to better manage their time du...,Custom Writing,"[Helpfulness, Length, Factuality, Language Style]",[And],And,"[{'point_id': 0, 'question': '模型回复的语言风格是否客观中立？..."


In [22]:
print(complex_bench.iloc[0]["instruction_en"])

Background:
""
Shending Technology (Nanjing) Co.,Ltd. (hereinafter referred to as "Shending Technology") has recently completed a new strategic financing round worth tens of millions of RMB. The investors are Shenzhen Hi-Tech Investment Group and Kunshan Taiwan Business Development Fund. 
The funds from this round will be allocated towards the expansion of solution development and mass promotion of smart sensing fusion chips.
Shending Technology boasts fully independent R&D capabilities in 3D smart sensing fusion technology. Currently, leveraging its extensive productization experience and supply chain integration capabilities, the company's R&D team has developed a 3D smart sensing fusion SoC chip platform based on Smart Fusion technology. This platform meets the unique scenario requirements of mobile robots and retains the comprehensive expandability of mobile robot hardware and offers cost advantages.
Regarding future development, a relevant representative from Shending Technology s

objectives:
- The pompt should extract the constraints existing in the instruction.
- The constraints needs to be atomic 


## Version 1

In [23]:
def prompt_extract_constraints(instruction):
    prompt = f"""
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Constraints are the additional requirements apart from the main task that the LLM needs to follow while generating the response.
    Rules for completing the task:
    1. The constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    2. The constraints should be extracted from the instruction and should not include any additional information.
    ## Input Format:
    {{"instruction": {instruction}
    }}
    ## Response Format:
    You should extract the constraints from the instruction and
    return them in a JSON format with the following structure:
    {{ "constraints": [<constraint1>, <constraint2>, ...] }}
    where each <constraint> is a string representing a single atomic constraint.
    """
    return prompt
prompt = prompt_extract_constraints(complex_bench.iloc[0]["instruction_en"])

In [24]:
response = client2.get_model_response(prompt)
print(response)

{
  "constraints": [
    "Rewrite the given background information into English news format.",
    "The news should consist of a 'headline' and a 'body'.",
    "The 'headline' should summarize the news, including the business, financing, and purpose of the financing.",
    "The 'body' should describe the enterprise and financing in the first paragraph, including financing progress, financier, amount, and purpose.",
    "Subsequent paragraphs should detail the enterprise's name, business, team, speeches, core technology, technology direction, and product progress.",
    "Ensure the headline and body content are consistent.",
    "Re-express information without changing names of people, objects, numbers, dates, or behaviors.",
    "Use English language for the rewritten news."
  ]
}


## Generate constraint method

In [25]:

def generate_constraints(instruction,prompt_function=prompt_extract_constraints):
    print("Generating constraints for instruction:\n",instruction)
    prompt = prompt_function(instruction)
    print("\nPrompt for constraint extraction:\n", prompt)
    response = client2.get_model_response(prompt)
    print("\ngenerated response:\n", response)
    constraints = extract_json(response, "constraints")
    return constraints

## Testing Followbench examples


In [26]:
followbench_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852 entries, 0 to 1851
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   example_id   1852 non-null   int64 
 1   category     1852 non-null   object
 2   source       1852 non-null   object
 3   instruction  1852 non-null   object
 4   level        1852 non-null   int64 
 5   target       1852 non-null   object
dtypes: int64(2), object(4)
memory usage: 86.9+ KB


In [27]:
ex1 = generate_constraints(followbench_data.iloc[0]["instruction"])

Generating constraints for instruction:
 Pick one category for the following text. The options are - company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work. Michael DenDekker -  Michael G. DenDekker (born July 11 1961) is an assemblyman for the state of New York's 34th district which includes the neighborhoods of Woodside Jackson Heights and East Elmhurst all in the borough/county of Queens.

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Constraints are the additional requirements apart from the main task that the LLM needs to follow while generating the response.
    Rules for completing the task:
    1. The constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    2. The constraints should be extracte


generated response:
 {
  "constraints": [
    "The response should pick one category",
    "The categories to choose from are: company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work",
    "The response should identify the category for Michael DenDekker"
  ]
}


In [ ]:
instruction = """  Identify one category from the list below for the input text, and also infer the sentiment (positive, neutral, or negative) conveyed in the text. Your options for the category are - company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film, or written work. Michael DenDekker - Michael G. DenDekker (born July 11 1961) is an assemblyman for the state of New York's 34th district which includes the neighborhoods of Woodside, Jackson Heights, and East Elmhurst, all in the borough/county of Queens.
"""

In [ ]:
instruction = """Generating constraints for instruction:
 Background:
""
Shending Technology (Nanjing) Co.,Ltd. (hereinafter referred to as "Shending Technology") has recently completed a new strategic financing round worth tens of millions of RMB. The investors are Shenzhen Hi-Tech Investment Group and Kunshan Taiwan Business Development Fund. 
The funds from this round will be allocated towards the expansion of solution development and mass promotion of smart sensing fusion chips.
Shending Technology boasts fully independent R&D capabilities in 3D smart sensing fusion technology. Currently, leveraging its extensive productization experience and supply chain integration capabilities, the company's R&D team has developed a 3D smart sensing fusion SoC chip platform based on Smart Fusion technology. This platform meets the unique scenario requirements of mobile robots and retains the comprehensive expandability of mobile robot hardware and offers cost advantages.
Regarding future development, a relevant representative from Shending Technology stated: "At present, Shending Technology has acquired multiple core technologies and patent achievements, and has accumulated technical advantages in sensing fusion, image algorithms, and artificial intelligence. Following this financing, Shending Technology will continue to adhere to innovation-driven development, advancing the smart sensing fusion field, and providing higher-quality solutions and services for the intelligence of perception systems in mobile robots, AR/VR augmented/virtual reality, and autonomous driving." 
""

Please rewrite it into English news based on the background information. The news consists of a "headline" and a "body", and the rewriting takes place in the body part.
The "headline" should summarise the news. The headline consists of the business and financing, and the purpose of the financing.
The "body" should describe the enterprise and financing in the first paragraph (including the progress of the financing, the financier, the amount of financing, and the purpose of financing); in the following paragraphs, it describes the enterprise in terms of its name, business, team, speeches, core technology, technology direction, and product progress, and so on.

Your news needs to comply with the following regulations:
1. The headline and the content of the main body must fit together and be consistent.
2. Re-express the information in the body of the background information, but the names of people, objects, numbers, dates, and behaviour can not be changed.
3. The language of the rewritten news should be in English.

Output format:
""
Today's news:
Title:
[ ]
Text:
[ ]
...
"""

In [28]:
ex2 = generate_constraints(followbench_data.iloc[1]["instruction"])

Generating constraints for instruction:
 Identify one category from the list below for the input text, and also infer the sentiment (positive, neutral, or negative) conveyed in the text. Your options for the category are - company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film, or written work. Michael DenDekker - Michael G. DenDekker (born July 11 1961) is an assemblyman for the state of New York's 34th district which includes the neighborhoods of Woodside, Jackson Heights, and East Elmhurst, all in the borough/county of Queens.

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Constraints are the additional requirements apart from the main task that the LLM needs to follow while generating the response.
    Rules for completing the task:
    1. The constraints should be 


generated response:
 {
  "constraints": [
    "Identify one category from the given list for the input text",
    "Infer the sentiment (positive, neutral, or negative) conveyed in the text",
    "The categories to choose from are: company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film, or written work",
    "The input text is about Michael DenDekker"
  ]
}


In [29]:
ex3 = generate_constraints(followbench_data.iloc[400]["instruction"])

Generating constraints for instruction:
 Could you please determine the airspeed velocity of an unladen swallow? Present your response by initially stating the bird species and concluding with the velocity figure. Please follow a hierarchical structure, starting with the bird’s Family and Genus before moving to Species and finally the airspeed velocity. 

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Constraints are the additional requirements apart from the main task that the LLM needs to follow while generating the response.
    Rules for completing the task:
    1. The constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    2. The constraints should be extracted from the instruction and should not include any additional information.
    ## Input Format:
    {"instruction": Could you please determine the airs


generated response:
 {
  "constraints": [
    "The response should start with the bird's Family and Genus.",
    "The response should then specify the bird's Species.",
    "The response should conclude with the airspeed velocity figure.",
    "The response should follow a hierarchical structure."
  ]
}


## ComplexBench

In [30]:
complex_bench.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150 entries, 0 to 1149
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   main_id                1150 non-null   int64 
 1   group                  1150 non-null   object
 2   idx_in_group           1150 non-null   int64 
 3   instruction            1150 non-null   object
 4   instruction_en         1150 non-null   object
 5   task_types             1150 non-null   object
 6   constraint_dimensions  1150 non-null   object
 7   composition_types      1150 non-null   object
 8   category               1150 non-null   object
 9   scoring_questions      1150 non-null   object
dtypes: int64(2), object(8)
memory usage: 90.0+ KB


In [31]:
ex1 = generate_constraints(complex_bench.iloc[0]["instruction_en"])


Generating constraints for instruction:
 Background:
""
Shending Technology (Nanjing) Co.,Ltd. (hereinafter referred to as "Shending Technology") has recently completed a new strategic financing round worth tens of millions of RMB. The investors are Shenzhen Hi-Tech Investment Group and Kunshan Taiwan Business Development Fund. 
The funds from this round will be allocated towards the expansion of solution development and mass promotion of smart sensing fusion chips.
Shending Technology boasts fully independent R&D capabilities in 3D smart sensing fusion technology. Currently, leveraging its extensive productization experience and supply chain integration capabilities, the company's R&D team has developed a 3D smart sensing fusion SoC chip platform based on Smart Fusion technology. This platform meets the unique scenario requirements of mobile robots and retains the comprehensive expandability of mobile robot hardware and offers cost advantages.
Regarding future development, a relevant 


generated response:
 {
  "constraints": [
    "Rewrite the given background information into English news format.",
    "The news should consist of a 'headline' and a 'body'.",
    "The 'headline' should summarize the news, including the business, financing, and purpose of the financing.",
    "The 'body' should describe the enterprise and financing in the first paragraph, including financing progress, financier, amount, and purpose.",
    "Subsequent paragraphs should detail the enterprise's name, business, team, speeches, core technology, technology direction, and product progress.",
    "Ensure the headline and body content are consistent.",
    "Re-express information without changing names of people, objects, numbers, dates, or behaviors.",
    "Use English language for the rewritten news."
  ]
}


In [32]:
ex2 = generate_constraints(complex_bench.iloc[1]["instruction_en"])

Generating constraints for instruction:
 You are a rapper, help me write a song lyric, with the first line being 'Always can't keep up with the pace.' The overall lyric should be positive and uplifting, containing a small amount of English words. Additionally, the context of the lyrics should be smooth, without any jumps in information or scenes.

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Constraints are the additional requirements apart from the main task that the LLM needs to follow while generating the response.
    Rules for completing the task:
    1. The constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    2. The constraints should be extracted from the instruction and should not include any additional information.
    ## Input Format:
    {"instruction": You are a rapper, help me write a song lyri


generated response:
 {
  "constraints": [
    "The response should be a song lyric",
    "The first line of the lyric should be 'Always can't keep up with the pace.'",
    "The overall tone of the lyric should be positive and uplifting",
    "The lyric should contain a small amount of English words",
    "The context of the lyrics should be smooth and coherent"
  ]
}


In [33]:
ex3 = generate_constraints(complex_bench.iloc[500]["instruction_en"])

Generating constraints for instruction:
 Write a novel about Armor Hero, where the protagonist is exceptionally gifted but loses the qualification to become an Armor Hero due to a setup, thus joining the antagonist camp. Requirement: 
- The novel must simultaneously include punctuation marks {！ ： ~}.

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Constraints are the additional requirements apart from the main task that the LLM needs to follow while generating the response.
    Rules for completing the task:
    1. The constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    2. The constraints should be extracted from the instruction and should not include any additional information.
    ## Input Format:
    {"instruction": Write a novel about Armor Hero, where the protagonist is exceptionally gifted but loses th


generated response:
 {
  "constraints": [
    "The novel must include the protagonist who is exceptionally gifted but loses qualification to become an Armor Hero due to a setup.",
    "The protagonist must join the antagonist camp after losing the qualification.",
    "The novel must simultaneously include the punctuation marks: '！' and '：' and '~'."
  ]
}


## For code generation

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   dataset                    60 non-null     object
 1   instruction                60 non-null     object
 2   code                       60 non-null     object
 3   test                       20 non-null     object
 4   Characteristics            60 non-null     object
 5   constraints                60 non-null     object
 6   Characteristics_List       60 non-null     object
 7   Mapped_Characteristics     60 non-null     object
 8   Mapped_Constraints         60 non-null     object
 9   Mapped_Characteristics_v2  60 non-null     object
 10  Mapped_Constraints_v2      60 non-null     object
 11  categories_v1              60 non-null     object
 12  categories_v2              60 non-null     object
 13  Manual_categories          60 non-null     object
 14  LLM_categori

In [76]:
instrcution = """
 Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

Example Input:
Point 1: (3, 4, 5)
Point 2: (2, 6, -1)

Example Output:
Distance: 2.45
Vector components: (1, 2, -6)
Angle with x-axis: 32.94 degrees
"""

generated response:
 {
  "constraints": [
    "The points are represented as tuples of three integers.",
    "The distance should be calculated in the 3-dimensional space.",
    "The distance should be rounded to two decimal places.",
    "The program should display the vector components of the resulting distance.",
    "The program should also calculate and display the angle between the vector components of the resulting distance and the x-axis.",
    "The angle should be rounded to two decimal places."
  ]
}

In [35]:
ex1 = generate_constraints(df.iloc[0]["instruction"])

Generating constraints for instruction:
 Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

Example Input:
Point 1: (3, 4, 5)
Point 2: (2, 6, -1)

Example Output:
Distance: 2.45
Vector components: (1, 2, -6)
Angle with x-axis: 32.94 degrees

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Constraints are the additional requirements apart from the main task that the LLM needs to follow while generating the response.
    Rules for completing the task:
    1. The constraints should be atomic, meaning each constraint should represent a single requirement or condition.


generated response:
 {
  "constraints": [
    "The points are represented as tuples of three integers.",
    "The distance should be calculated in the 3-dimensional space.",
    "The distance should be rounded to two decimal places.",
    "The program should display the vector components of the resulting distance.",
    "The program should also calculate and display the angle between the vector components of the resulting distance and the x-axis.",
    "The angle should be rounded to two decimal places."
  ]
}


## Version 2

In [36]:
def prompt_extract_constraints_v2(instruction):
    prompt = f"""
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Rules for completing the task:
    1. Extracted constraints should not include core problem to solve, any explanations of the problem or the inputs and input examples.
    2. constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    3. Split Compound Constraints into Atomic Ones: If any sentence or clause contains multiple distinct constraints, split it into multiple entries, ensuring each extracted constraint refers to exactly one requirement or condition. Do not combine multiple checks (e.g., multiple conditions separated by "and" or "or" clauses) into one constraint.
    4. There should be a clear distinction between the core problem description and the constraints.
    5. The core problem description should not be present in the constraints extraction.
    ## Input Format:
    {{"instruction": {instruction}
    }}
    ## Response Format:
    You should extract the constraints from the instruction and
    return them in a JSON format with the following structure:
    {{ "constraints": [<constraint1>, <constraint2>, ...] }}
    where each <constraint> is a string representing a single atomic constraint.
    """
    return prompt
prompt = prompt_extract_constraints_v2(df.iloc[0]["instruction"])

In [77]:
ex3 = generate_constraints(complex_bench.iloc[5]["instruction_en"], prompt_function=prompt_extract_constraints_v2)

Generating constraints for instruction:
 You are a top student majoring in finance and accounting. Please publish an article in the style of Zhihu about the growth secrets for finance and accounting professionals centered around CPA, ACCA, and other exams. Subtly insert an advertisement for the ACCA exam at the end. The text should be positive, uplifting, and attractive.

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Rules for completing the task:
    1. Extracted constraints should not include core problem to solve, any explanations of the problem or the inputs and input examples.
    2. constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    3. Split Compound Constraints into Atomic Ones: If any sentence or clause contains multiple distinct constraints, split it into multiple entries, ensuring each extracted 

In [ ]:
instruction = """
 Now you are Archduke Ferdinand. After fully understanding the experiences and personality of this character, use a rigorous and formal language style but with an impatient tone, and use the style of European and American translation language, to answer my question. My question is: Why do you advocate the annexation of the Kingdom of Serbia? Then, based on the emotional inclination of your answer, ask me a question related to the social environment of the Austro-Hungarian Empire at the time. You need to always play the designated character, and your answers should align with the given setting.
"""

generated response:
 {
  "constraints": [
    "Generate response as Archduke Ferdinand",
    "Use rigorous and formal language style",
    "Maintain an impatient tone",
    "Use the style of European and American translation language",
    "Answer the question: 'Why do you advocate the annexation of the Kingdom of Serbia?'",
    "Align response with the given character setting",
    "Ask a question related to the social environment of the Austro-Hungarian Empire based on the emotional inclination of the answer",
    "Always play the designated character",
    "Ensure answers align with the given setting"
  ]
}

In [78]:
ex3 = generate_constraints(complex_bench.iloc[15]["instruction_en"], prompt_function=prompt_extract_constraints_v2)

Generating constraints for instruction:
 Now you are Archduke Ferdinand. After fully understanding the experiences and personality of this character, use a rigorous and formal language style but with an impatient tone, and use the style of European and American translation language, to answer my question. My question is: Why do you advocate the annexation of the Kingdom of Serbia? Then, based on the emotional inclination of your answer, ask me a question related to the social environment of the Austro-Hungarian Empire at the time. You need to always play the designated character, and your answers should align with the given setting.

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Rules for completing the task:
    1. Extracted constraints should not include core problem to solve, any explanations of the problem or the inputs and input examples.
    2. constraints s

In [37]:
ex1 = generate_constraints(df.iloc[0]["instruction"],prompt_function=prompt_extract_constraints_v2)


Generating constraints for instruction:
 Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

Example Input:
Point 1: (3, 4, 5)
Point 2: (2, 6, -1)

Example Output:
Distance: 2.45
Vector components: (1, 2, -6)
Angle with x-axis: 32.94 degrees

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Rules for completing the task:
    1. Extracted constraints should not include core problem to solve, any explanations of the problem or the inputs and input examples.
    2. constraints should be atomic, meaning each constraint should represent a single requirement or condition.


generated response:
 {
  "constraints": [
    "The program should calculate the distance between two points in 3-dimensional space.",
    "The points are to be represented as tuples of three integers.",
    "The calculated distance should be rounded to two decimal places.",
    "The program should also determine the vector components of the distance.",
    "The program should calculate the angle between the vector components of the distance and the x-axis.",
    "The angle should be rounded to two decimal places and displayed in degrees."
  ]
}


generated response:
 {
  "constraints": [
    "The program should calculate the distance between two points in 3-dimensional space.",
    "The points are to be represented as tuples of three integers.",
    "The calculated distance should be rounded to two decimal places.",
    "The program should also determine the vector components of the distance.",
    "The program should calculate the angle between the vector components of the distance and the x-axis.",
    "The angle should be rounded to two decimal places and displayed in degrees."
  ]
}

In [38]:
ex1 = generate_constraints(df.iloc[0]["instruction"],prompt_function=prompt_extract_constraints)


Generating constraints for instruction:
 Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

Example Input:
Point 1: (3, 4, 5)
Point 2: (2, 6, -1)

Example Output:
Distance: 2.45
Vector components: (1, 2, -6)
Angle with x-axis: 32.94 degrees

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Constraints are the additional requirements apart from the main task that the LLM needs to follow while generating the response.
    Rules for completing the task:
    1. The constraints should be atomic, meaning each constraint should represent a single requirement or condition.


generated response:
 {
  "constraints": [
    "The points are represented as tuples of three integers.",
    "The distance should be calculated in the 3-dimensional space.",
    "The distance should be rounded to two decimal places.",
    "The program should display the vector components of the resulting distance.",
    "The program should also calculate and display the angle between the vector components of the resulting distance and the x-axis.",
    "The angle should be rounded to two decimal places."
  ]
}


generated response:
 {
  "constraints": [
    "Identify one category from the provided list",
    "Infer the sentiment (positive, neutral, or negative) conveyed in the text",
    "Category options: company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film, or written work",
    "Input text refers to: Michael DenDekker",
    "Michael DenDekker's role: assemblyman for the state of New York's 34th district",
    "Neighborhoods included in district 34: Woodside, Jackson Heights, East Elmhurst",
    "Borough/county of district 34: Queens",
    "Date of birth: July 11, 1961"
  ]
}

In [39]:
ex2 = generate_constraints(followbench_data.iloc[1]["instruction"],prompt_function=prompt_extract_constraints_v2)

Generating constraints for instruction:
 Identify one category from the list below for the input text, and also infer the sentiment (positive, neutral, or negative) conveyed in the text. Your options for the category are - company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film, or written work. Michael DenDekker - Michael G. DenDekker (born July 11 1961) is an assemblyman for the state of New York's 34th district which includes the neighborhoods of Woodside, Jackson Heights, and East Elmhurst, all in the borough/county of Queens.

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Rules for completing the task:
    1. Extracted constraints should not include core problem to solve, any explanations of the problem or the inputs and input examples.
    2. constraints should be 


generated response:
 {
  "constraints": [
    "Identify one category from the provided list",
    "Infer the sentiment (positive, neutral, or negative) conveyed in the text",
    "Category options: company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film, or written work",
    "Input text refers to: Michael DenDekker",
    "Michael DenDekker's role: assemblyman for the state of New York's 34th district",
    "Neighborhoods included in district 34: Woodside, Jackson Heights, East Elmhurst",
    "Borough/county of district 34: Queens",
    "Date of birth: July 11, 1961"
  ]
}


In [40]:
ex3 = generate_constraints(complex_bench.iloc[0]["instruction_en"],prompt_function=prompt_extract_constraints_v2)

Generating constraints for instruction:
 Background:
""
Shending Technology (Nanjing) Co.,Ltd. (hereinafter referred to as "Shending Technology") has recently completed a new strategic financing round worth tens of millions of RMB. The investors are Shenzhen Hi-Tech Investment Group and Kunshan Taiwan Business Development Fund. 
The funds from this round will be allocated towards the expansion of solution development and mass promotion of smart sensing fusion chips.
Shending Technology boasts fully independent R&D capabilities in 3D smart sensing fusion technology. Currently, leveraging its extensive productization experience and supply chain integration capabilities, the company's R&D team has developed a 3D smart sensing fusion SoC chip platform based on Smart Fusion technology. This platform meets the unique scenario requirements of mobile robots and retains the comprehensive expandability of mobile robot hardware and offers cost advantages.
Regarding future development, a relevant 


generated response:
 {
  "constraints": [
    "The news must consist of a 'headline' and a 'body'.",
    "The 'headline' should summarize the news, including the business, financing, and purpose of the financing.",
    "The 'body' should describe the enterprise and financing in the first paragraph, including financing progress, financier, amount, and purpose.",
    "Subsequent paragraphs should detail the enterprise's name, business, team, speeches, core technology, technology direction, and product progress.",
    "The headline and content of the main body must be consistent.",
    "Information in the body of the background should be re-expressed but unchanged in terms of names, objects, numbers, dates, and behaviors.",
    "The language of the rewritten news must be English."
  ]
}


## Version 3 

In [89]:
def prompt_extract_constraints_v3(instruction):
    prompt = f"""
    context: You are given an instruction for a code generation task.
    Your Role:
    Extract only implementation constraints from the instruction.
    ## DO NOT:
    - DO NOT include the core problem to be solved.
    - DO NOT include problem description, input/output formats, examples, or explanations.
    - DO NOT include data representation details (e.g., “tuples of three integers”).
    - DO NOT include what to compute (e.g., “calculate the distance”).
    - DO NOT create constraints that are not explicitly mentioned in the instruction.
    ## DO:
    - Only extract implementation-specific requirements that constrain how the solution must be written or how results must be presented.
    - Split any compound constraints into atomic constraints.
    ## Input:
    {{
    "instruction": "{instruction}"
    }}
    ## Output Format:
    Return constraints as a JSON object in this format:
    {{
      "constraints": [
        "<atomic constraint 1>",
        "<atomic constraint 2>",
        ...
      ]
    }}
    Only return the JSON. Do not explain your reasoning or repeat the instruction.
    """
    return prompt


In [91]:
ex1 = generate_constraints(df.iloc[0]["instruction"], prompt_function=prompt_extract_constraints_v3)

Generating constraints for instruction:
 Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

Example Input:
Point 1: (3, 4, 5)
Point 2: (2, 6, -1)

Example Output:
Distance: 2.45
Vector components: (1, 2, -6)
Angle with x-axis: 32.94 degrees

Prompt for constraint extraction:
 
    context: You are given an instruction for a code generation task.
    Your Role:
    Extract only implementation constraints from the instruction.
    ## DO NOT:
    - DO NOT include the core problem to be solved.
    - DO NOT include problem description, input/output formats, examples, or explanations.
    - DO NOT include data representation details (e.g., “tuples of three integers”).
    - DO NOT include

generated response:
 {
  "constraints": [
    "The points are represented as tuples of three integers.",
    "The distance should be rounded to two decimal places.",
    "The program should display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places."
  ]
}

In [ ]:
ex1 = generate_constraints(df.iloc[0]["instruction"], prompt_function=prompt_extract_constraints_v3)

Generating constraints for instruction:
 Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

Example Input:
Point 1: (3, 4, 5)
Point 2: (2, 6, -1)

Example Output:
Distance: 2.45
Vector components: (1, 2, -6)
Angle with x-axis: 32.94 degrees

Prompt for constraint extraction:
 
    context: You are given an instruction for a code generation task.
    Your Role:
    Extract only implementation constraints from the instruction.
    ## DO NOT:
    - DO NOT include the core problem to be solved.
    - DO NOT include problem description, input/output formats, examples, or explanations.
    - DO NOT include data representation details (e.g., “tuples of three integers”).
    - DO NOT include

generated response:
 {
  "constraints": [
    "The distance between two points in 3D space must be calculated.",
    "Points are represented as tuples of three integers.",
    "The distance must be rounded to two decimal places.",
    "The program must display the vector components of the distance.",
    "The angle between the vector components of the distance and the x-axis must be calculated and displayed, rounded to two decimal places."
  ]
}

In [98]:
ex1 = generate_constraints_v2(df.iloc[0]["instruction"], prompt_function=prompt_extract_constraints_v3)

Generating constraints for instruction:
 Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

Example Input:
Point 1: (3, 4, 5)
Point 2: (2, 6, -1)

Example Output:
Distance: 2.45
Vector components: (1, 2, -6)
Angle with x-axis: 32.94 degrees



generated response:
 {
  "constraints": [
    "The distance between two points in 3D space must be calculated.",
    "Points are represented as tuples of three integers.",
    "The distance must be rounded to two decimal places.",
    "The program must display the vector components of the distance.",
    "The angle between the vector components of the distance and the x-axis must be calculated and displayed, rounded to two decimal places."
  ]
}


In [ ]:
ex2 = generate_constraints_v2(followbench_data.iloc[1]["instruction"],prompt_function=prompt_extract_constraints_v3)

Generating constraints for instruction:
 Identify one category from the list below for the input text, and also infer the sentiment (positive, neutral, or negative) conveyed in the text. Your options for the category are - company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film, or written work. Michael DenDekker - Michael G. DenDekker (born July 11 1961) is an assemblyman for the state of New York's 34th district which includes the neighborhoods of Woodside, Jackson Heights, and East Elmhurst, all in the borough/county of Queens.



generated response:
 {
  "constraints": [
    "Only extract one category from the provided list.",
    "Infer the sentiment (positive, neutral, or negative) from the text.",
    "The text is a description of a person, specifically a political figure.",
    "The output should include the category 'office holder' for the given text."
  ]
}


In [59]:
ex2 = generate_constraints(followbench_data.iloc[1]["instruction"],prompt_function=prompt_extract_constraints_v3)

Generating constraints for instruction:
 Identify one category from the list below for the input text, and also infer the sentiment (positive, neutral, or negative) conveyed in the text. Your options for the category are - company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film, or written work. Michael DenDekker - Michael G. DenDekker (born July 11 1961) is an assemblyman for the state of New York's 34th district which includes the neighborhoods of Woodside, Jackson Heights, and East Elmhurst, all in the borough/county of Queens.

Prompt for constraint extraction:
 
context: You are given an instruction for a code generation task.

Your Role:
Extract only implementation constraints from the instruction.

## DO NOT:
- DO NOT include the core problem to be solved.
- DO NOT include problem description, input/output formats, examples, or explanations.
- DO NOT include data representation details


generated response:
 {
  "constraints": [
    "Only extract one category from the provided list.",
    "Infer the sentiment (positive, neutral, or negative) from the text.",
    "The text is a description of a person, specifically a political figure.",
    "The output should include the category 'office holder' for the given text."
  ]
}


In [60]:
ex3 = generate_constraints(followbench_data.iloc[400]["instruction"],prompt_function=prompt_extract_constraints_v2)

Generating constraints for instruction:
 Could you please determine the airspeed velocity of an unladen swallow? Present your response by initially stating the bird species and concluding with the velocity figure. Please follow a hierarchical structure, starting with the bird’s Family and Genus before moving to Species and finally the airspeed velocity. 

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Rules for completing the task:
    1. Extracted constraints should not include core problem to solve, any explanations of the problem or the inputs and input examples.
    2. constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    3. Split Compound Constraints into Atomic Ones: If any sentence or clause contains multiple distinct constraints, split it into multiple entries, ensuring each extracted constraint refers


generated response:
 {
  "constraints": [
    "Present the response by initially stating the bird species.",
    "Conclude the response with the velocity figure.",
    "Follow a hierarchical structure.",
    "Start with the bird’s Family and Genus.",
    "Move to Species next.",
    "Finally, provide the airspeed velocity."
  ]
}


In [61]:
ex3 = generate_constraints(followbench_data.iloc[400]["instruction"],prompt_function=prompt_extract_constraints_v2)

Generating constraints for instruction:
 Could you please determine the airspeed velocity of an unladen swallow? Present your response by initially stating the bird species and concluding with the velocity figure. Please follow a hierarchical structure, starting with the bird’s Family and Genus before moving to Species and finally the airspeed velocity. 

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Rules for completing the task:
    1. Extracted constraints should not include core problem to solve, any explanations of the problem or the inputs and input examples.
    2. constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    3. Split Compound Constraints into Atomic Ones: If any sentence or clause contains multiple distinct constraints, split it into multiple entries, ensuring each extracted constraint refers


generated response:
 {
  "constraints": [
    "Present the response by initially stating the bird species.",
    "Conclude the response with the velocity figure.",
    "Follow a hierarchical structure.",
    "Start with the bird’s Family and Genus.",
    "Move to Species next.",
    "Finally, provide the airspeed velocity."
  ]
}


## On complexbench

generated response:
 {
  "constraints": [
    "Use a rigorous and formal language style",
    "Maintain an impatient tone",
    "Utilize the style of European and American translation language",
    "Align answers with the character of Archduke Ferdinand",
    "Ensure answers are consistent with the given setting",
    "Ask a question related to the social environment of the Austro-Hungarian Empire based on the emotional inclination of the answer"
  ]
}

In [94]:
ex1 = generate_constraints(complex_bench.iloc[15]["instruction_en"], prompt_function=prompt_extract_constraints_v3)

Generating constraints for instruction:
 Now you are Archduke Ferdinand. After fully understanding the experiences and personality of this character, use a rigorous and formal language style but with an impatient tone, and use the style of European and American translation language, to answer my question. My question is: Why do you advocate the annexation of the Kingdom of Serbia? Then, based on the emotional inclination of your answer, ask me a question related to the social environment of the Austro-Hungarian Empire at the time. You need to always play the designated character, and your answers should align with the given setting.

Prompt for constraint extraction:
 
    context: You are given an instruction for a code generation task.
    Your Role:
    Extract only implementation constraints from the instruction.
    ## DO NOT:
    - DO NOT include the core problem to be solved.
    - DO NOT include problem description, input/output formats, examples, or explanations.
    - DO NOT 


generated response:
 {
  "constraints": [
    "Use a rigorous and formal language style",
    "Maintain an impatient tone",
    "Utilize the style of European and American translation language",
    "Align answers with the character of Archduke Ferdinand",
    "Ensure answers are consistent with the given setting",
    "Ask a question related to the social environment of the Austro-Hungarian Empire based on the emotional inclination of the answer"
  ]
}


In [63]:
ex1 = generate_constraints(complex_bench.iloc[0]["instruction_en"], prompt_function=prompt_extract_constraints_v3)

Generating constraints for instruction:
 Background:
""
Shending Technology (Nanjing) Co.,Ltd. (hereinafter referred to as "Shending Technology") has recently completed a new strategic financing round worth tens of millions of RMB. The investors are Shenzhen Hi-Tech Investment Group and Kunshan Taiwan Business Development Fund. 
The funds from this round will be allocated towards the expansion of solution development and mass promotion of smart sensing fusion chips.
Shending Technology boasts fully independent R&D capabilities in 3D smart sensing fusion technology. Currently, leveraging its extensive productization experience and supply chain integration capabilities, the company's R&D team has developed a 3D smart sensing fusion SoC chip platform based on Smart Fusion technology. This platform meets the unique scenario requirements of mobile robots and retains the comprehensive expandability of mobile robot hardware and offers cost advantages.
Regarding future development, a relevant 


generated response:
 {
  "constraints": [
    "The output must be in English.",
    "The news must have a headline and a body.",
    "The headline should summarize the news, including business, financing, and purpose.",
    "The body's first paragraph must describe the enterprise and financing details.",
    "Subsequent paragraphs should detail the enterprise's name, business, team, speeches, core technology, technology direction, and product progress.",
    "Names of people, objects, numbers, dates, and behaviors must remain unchanged.",
    "The language in the rewritten news must be consistent with the headline and background information."
  ]
}


## Final analysis

In [64]:
def generate_constraints_v2(instruction, prompt_function=prompt_extract_constraints_v3):
    print("Generating constraints for instruction:\n", instruction)
    prompt = prompt_function(instruction)
    response = client2.get_model_response(prompt)
    print("\ngenerated response:\n", response)
    constraints = extract_json(response, "constraints")
    return constraints

## Code Generation

In [65]:
ex1 = generate_constraints_v2(df.iloc[0]["instruction"])
ex2 = generate_constraints_v2(df.iloc[1]["instruction"], prompt_function=prompt_extract_constraints_v2)
ex3 = generate_constraints_v2(df.iloc[0]["instruction"], prompt_function=prompt_extract_constraints_v3)

Generating constraints for instruction:
 Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

Example Input:
Point 1: (3, 4, 5)
Point 2: (2, 6, -1)

Example Output:
Distance: 2.45
Vector components: (1, 2, -6)
Angle with x-axis: 32.94 degrees



generated response:
 {
  "constraints": [
    "The solution must be implemented as a program.",
    "The program must accept two tuples of three integers as input.",
    "The program must calculate the Euclidean distance between the two points and round it to two decimal places.",
    "The program must compute the vector components of the distance.",
    "The program must calculate the angle between the vector components and the x-axis, rounding it to two decimal places.",
    "The program must display the distance and angle in the specified format."
  ]
}
Generating constraints for instruction:
 Assuming that the given sentence is stored in a variable "sentence" in a Python program, how can the program improve it by adding a descriptive tag to the word "apples" using HTML?
For example, if "apples" are categorized as "fruit", the program can add the HTML tag <span class="fruit">apples</span> to the sentence. How can this be achieved using Python and HTML?

generated response:
 {
  "co

## ComplexBench

In [66]:
ex1 = generate_constraints_v2(complex_bench.iloc[0]["instruction_en"])
ex2 = generate_constraints_v2(complex_bench.iloc[1]["instruction_en"], prompt_function=prompt_extract_constraints_v2)
ex3 = generate_constraints_v2(complex_bench.iloc[0]["instruction_en"], prompt_function=prompt_extract_constraints_v3)

Generating constraints for instruction:
 Background:
""
Shending Technology (Nanjing) Co.,Ltd. (hereinafter referred to as "Shending Technology") has recently completed a new strategic financing round worth tens of millions of RMB. The investors are Shenzhen Hi-Tech Investment Group and Kunshan Taiwan Business Development Fund. 
The funds from this round will be allocated towards the expansion of solution development and mass promotion of smart sensing fusion chips.
Shending Technology boasts fully independent R&D capabilities in 3D smart sensing fusion technology. Currently, leveraging its extensive productization experience and supply chain integration capabilities, the company's R&D team has developed a 3D smart sensing fusion SoC chip platform based on Smart Fusion technology. This platform meets the unique scenario requirements of mobile robots and retains the comprehensive expandability of mobile robot hardware and offers cost advantages.
Regarding future development, a relevant 


generated response:
 {
  "constraints": [
    "The output must be in English.",
    "The news must have a headline and a body.",
    "The headline should summarize the news, including business, financing, and purpose.",
    "The body's first paragraph must describe the enterprise and financing details.",
    "Subsequent paragraphs should detail the enterprise's name, business, team, speeches, core technology, technology direction, and product progress.",
    "Names of people, objects, numbers, dates, and behaviors must remain unchanged.",
    "The language in the rewritten news must be consistent with the headline and background information."
  ]
}
Generating constraints for instruction:
 You are a rapper, help me write a song lyric, with the first line being 'Always can't keep up with the pace.' The overall lyric should be positive and uplifting, containing a small amount of English words. Additionally, the context of the lyrics should be smooth, without any jumps in information or s

## Followbench

In [67]:
ex1 = generate_constraints(followbench_data.iloc[0]["instruction"])
ex2 = generate_constraints(followbench_data.iloc[1]["instruction"],prompt_function=prompt_extract_constraints_v2)
ex3 = generate_constraints(followbench_data.iloc[400]["instruction"],prompt_function=prompt_extract_constraints_v3)

Generating constraints for instruction:
 Pick one category for the following text. The options are - company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work. Michael DenDekker -  Michael G. DenDekker (born July 11 1961) is an assemblyman for the state of New York's 34th district which includes the neighborhoods of Woodside Jackson Heights and East Elmhurst all in the borough/county of Queens.

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Constraints are the additional requirements apart from the main task that the LLM needs to follow while generating the response.
    Rules for completing the task:
    1. The constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    2. The constraints should be extracte


generated response:
 {
  "constraints": [
    "The response should pick one category for the given text.",
    "The categories to choose from are: company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work.",
    "The response should provide information about Michael G. DenDekker.",
    "The response should include the district he represents in the New York State Assembly.",
    "The response should list the neighborhoods included in his district."
  ]
}
Generating constraints for instruction:
 Identify one category from the list below for the input text, and also infer the sentiment (positive, neutral, or negative) conveyed in the text. Your options for the category are - company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film, or written work. Michael DenDekker - Michael G. DenDekker

# Version 4 Few shot Propmting

In [80]:
def prompt_extract_constraints_v4(instruction):
    prompt = f"""
context: You are given an instruction for a code generation task.

Your Role:
Extract only implementation constraints from the instruction.

## DO NOT:
- DO NOT include the core problem to be solved.
- DO NOT include problem description, input/output formats, examples, or explanations.
- DO NOT include data representation details (e.g., “tuples of three integers”).
- DO NOT include what to compute (e.g., “calculate the distance”).
- DO NOT create constraints that are not explicitly mentioned in the instruction.

## DO:
- Only extract implementation-specific requirements that constrain how the solution must be written or how results must be presented.
- Split any compound constraints into atomic constraints.

## Few-shot Examples:

### Example 1
Instruction:
Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

Example Input:
Point 1: (3, 4, 5)
Point 2: (2, 6, -1)

Example Output:
Distance: 2.45  
Vector components: (1, 2, -6)  
Angle with x-axis: 32.94 degrees

Output:
{{
  "constraints": [
    "Return the distance rounded to two decimal places.",
    "Display the vector components as a tuple of three integers.",
    "Return the angle with the x-axis rounded to two decimal places in degrees."
  ]
}}

---

## Input:
{{
"instruction": "{instruction}"
}}

## Output Format:
Return constraints as a JSON object in this format:
{{
  "constraints": [
    "<atomic constraint 1>",
    "<atomic constraint 2>",
    ...
  ]
}}

Only return the JSON. Do not explain your reasoning or repeat the instruction.
"""
    return prompt
ex1 = generate_constraints_v2(df.iloc[20]["instruction"], prompt_function=prompt_extract_constraints_v4)

Generating constraints for instruction:
 Rollback the update of a directory by restoring it from a backup. Constants: - BACKUP_DIR: The directory where backups are stored. Default is '/tmp/backup'. >>> task_func('/tmp/nonexistent') 'Backup directory /tmp/backup does not exist. Cannot rollback update.'
Note that: This function will return the restored directory path on successful rollback, or an error message otherwise.
The function should output with:
    directory (str): The restored directory path if successful, otherwise an error message.
You should write self-contained code starting with:
```
import os
import shutil
# Constants
BACKUP_DIR = '/tmp/backup'
def task_func(directory):
```



generated response:
 {
  "constraints": [
    "Import the 'os' and 'shutil' modules.",
    "Define a constant 'BACKUP_DIR' with the value '/tmp/backup'.",
    "The function 'task_func' must accept a single argument 'directory'.",
    "The function should return the restored directory path on successful rollback, or an error message otherwise.",
    "The function output should be formatted as a string."
  ]
}


In [95]:
ex1 = generate_constraints(complex_bench.iloc[15]["instruction_en"], prompt_function=prompt_extract_constraints_v4)

Generating constraints for instruction:
 Now you are Archduke Ferdinand. After fully understanding the experiences and personality of this character, use a rigorous and formal language style but with an impatient tone, and use the style of European and American translation language, to answer my question. My question is: Why do you advocate the annexation of the Kingdom of Serbia? Then, based on the emotional inclination of your answer, ask me a question related to the social environment of the Austro-Hungarian Empire at the time. You need to always play the designated character, and your answers should align with the given setting.

Prompt for constraint extraction:
 
context: You are given an instruction for a code generation task.

Your Role:
Extract only implementation constraints from the instruction.

## DO NOT:
- DO NOT include the core problem to be solved.
- DO NOT include problem description, input/output formats, examples, or explanations.
- DO NOT include data representatio

In [96]:
def prompt_extract_constraints_v5(instruction):
    prompt = f"""
context: You are given an instruction for a code generation task.

Your Role:
Extract only implementation constraints from the instruction.

## DO NOT:
- DO NOT include the core problem to be solved.
- DO NOT include problem description, input/output formats, examples, or explanations.
- DO NOT include data representation details (e.g., “tuples of three integers”).
- DO NOT include what to compute (e.g., “calculate the distance”).
- DO NOT create constraints that are not explicitly mentioned in the instruction.

## DO:
- Only extract implementation-specific requirements that constrain how the solution must be written or how results must be presented.
- Split any compound constraints into atomic constraints.

## Few-shot Example:

### Instruction:
Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

### Example Input:
Point 1: (3, 4, 5)  
Point 2: (2, 6, -1)

### Example Output:
Distance: 2.45  
Vector components: (1, 2, -6)  
Angle with x-axis: 32.94 degrees

### Incorrect Response:
{{
  "constraints": [
    "Use tuples of three integers to represent points in 3D space.",
    "Distance between points must be rounded to two decimal places.",
    "Program must display the vector components of the distance.",
    "Angle between the vector components of the distance and the x-axis must be calculated and rounded to two decimal places in degrees."
  ]
}}

(Explanation: This response includes representation details and rephrases what to compute, which violates the instructions.)

### Correct Response:
{{
  "constraints": [
    "Return the distance rounded to two decimal places.",
    "Display the vector components as a tuple of three integers.",
    "Return the angle with the x-axis rounded to two decimal places in degrees."
  ]
}}
## Input:
{{
"instruction": "{instruction}"
}}

## Output Format:
Return constraints as a JSON object in this format:
{{
  "constraints": [
    "<atomic constraint 1>",
    "<atomic constraint 2>",
    ...
  ]
}}

Only return the JSON. Do not explain your reasoning or repeat the instruction.
"""
    return prompt


In [97]:
ex1 = generate_constraints(complex_bench.iloc[15]["instruction_en"], prompt_function=prompt_extract_constraints_v5)

Generating constraints for instruction:
 Now you are Archduke Ferdinand. After fully understanding the experiences and personality of this character, use a rigorous and formal language style but with an impatient tone, and use the style of European and American translation language, to answer my question. My question is: Why do you advocate the annexation of the Kingdom of Serbia? Then, based on the emotional inclination of your answer, ask me a question related to the social environment of the Austro-Hungarian Empire at the time. You need to always play the designated character, and your answers should align with the given setting.

Prompt for constraint extraction:
 
context: You are given an instruction for a code generation task.

Your Role:
Extract only implementation constraints from the instruction.

## DO NOT:
- DO NOT include the core problem to be solved.
- DO NOT include problem description, input/output formats, examples, or explanations.
- DO NOT include data representatio

## Trying out GPT-4omini

In [69]:
import pandas as pd
import time
from openai import OpenAI
import os

if os.path.exists("/content"):
    from google.colab import userdata
    openai_api_key = userdata.get('OPENAPI_KEY')
else:
    from dotenv import load_dotenv
    load_dotenv()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    print("Running in local environment, loaded key from .env")

# Check if key was actually found
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set. Please check your environment or .env file.")

# Initialize OpenAI client
client = OpenAI(api_key=openai_api_key)


Running in local environment, loaded key from .env


In [70]:
SYSTEM_PROMPT = f"""
You are a helpful assistant. You will be given a programming instruction and the corresponding code.
"""

In [71]:
import time
def get_response(user_prompt,system_prompt=SYSTEM_PROMPT, max_retries=1):
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
            )
            # print(response)
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error on attempt {attempt+1}: {e}")
            time.sleep(2)
    return "[]"

In [72]:
def generate_constraints_v3(instruction, prompt_function=prompt_extract_constraints_v3):
    print("Generating constraints for instruction:\n", instruction)
    prompt = prompt_function(instruction)
    response = get_response(prompt)
    print("\ngenerated response:\n", response)
    constraints = extract_json(response, "constraints")
    return constraints

In [73]:
ex1 = generate_constraints_v3(df.iloc[0]["instruction"])

Generating constraints for instruction:
 Calculate the distance between two points located in the 3-dimensional space. The points are represented as tuples of three integers. The distance should be rounded to two decimal places. Additionally, the program should also display the angle between the vector components of the resulting distance and the x-axis in degrees, rounded to two decimal places.

Example Input:
Point 1: (3, 4, 5)
Point 2: (2, 6, -1)

Example Output:
Distance: 2.45
Vector components: (1, 2, -6)
Angle with x-axis: 32.94 degrees



generated response:
 {
  "constraints": [
    "The distance should be rounded to two decimal places.",
    "The angle should be displayed in degrees.",
    "The angle should be rounded to two decimal places."
  ]
}


In [74]:
ex = generate_constraints(followbench_data.iloc[0]["instruction"])

Generating constraints for instruction:
 Pick one category for the following text. The options are - company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work. Michael DenDekker -  Michael G. DenDekker (born July 11 1961) is an assemblyman for the state of New York's 34th district which includes the neighborhoods of Woodside Jackson Heights and East Elmhurst all in the borough/county of Queens.

Prompt for constraint extraction:
 
    context: I have instruction for LLM to generate response.
    Task: Take the instrcution and extract the constraints from it.
    Constraints are the additional requirements apart from the main task that the LLM needs to follow while generating the response.
    Rules for completing the task:
    1. The constraints should be atomic, meaning each constraint should represent a single requirement or condition.
    2. The constraints should be extracte


generated response:
 {
  "constraints": [
    "The response should pick one category for the given text.",
    "The categories to choose from are: company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work.",
    "The response should provide information about Michael G. DenDekker.",
    "The response should include the district he represents in the New York State Assembly.",
    "The response should list the neighborhoods included in his district."
  ]
}
